# Nuclio - Training function

## Environment

In [1]:
# nuclio: ignore
import nuclio

### Configurations

In [2]:
%%nuclio config

# Trigger
spec.triggers.retrain.kind = "cron"
spec.triggers.retrain.attributes.interval = "1h"

# Base image
spec.build.baseImage = "python:3.6-jessie"

%nuclio: setting spec.triggers.retrain.kind to 'cron'
%nuclio: setting spec.triggers.retrain.attributes.interval to '1h'
%nuclio: setting spec.build.baseImage to 'python:3.6-jessie'


### Commands

In [14]:
%%nuclio cmd

############
# installs #
############

# Utils
pip install pyyaml
pip install pyarrow
pip install pandas

# Igz DB
pip install v3io_frames --upgrade

# Function
pip install scikit-learn==0.20.1
pip install xgboost --upgrade
pip install dask["complete"] --upgrade
pip install dask-ml["complete"] --upgrade

Invalid -W option ignored: invalid action: 'yaml.YAMLLoadWarning'
Invalid -W option ignored: invalid action: 'yaml.YAMLLoadWarning'
Invalid -W option ignored: invalid action: 'yaml.YAMLLoadWarning'
Invalid -W option ignored: invalid action: 'yaml.YAMLLoadWarning'
Requirement already up-to-date: v3io_frames in /User/.pythonlibs/lib/python3.6/site-packages (0.5.3)
Invalid -W option ignored: invalid action: 'yaml.YAMLLoadWarning'
    100% |################################| 5.4MB 9.9MB/s 
  Found existing installation: scikit-learn 0.20.2
    Uninstalling scikit-learn-0.20.2:
      Successfully uninstalled scikit-learn-0.20.2
Requirement already up-to-date: xgboost in /User/.pythonlibs/lib/python3.6/site-packages (0.82)
Invalid -W option ignored: invalid action: 'yaml.YAMLLoadWarning'
Requirement already up-to-date: dask[complete] in /User/.pythonlibs/lib/python3.6/site-packages (1.2.2)
Invalid -W option ignored: invalid action: 'yaml.YAMLLoadWarning'
Requirement already up-to-date: dask-m

### Variables

In [4]:
%%nuclio env

# DB Config
V3IO_FRAMESD=${V3IO_FRAMESD}
V3IO_USERNAME=${V3IO_USERNAME}
V3IO_ACCESS_KEY=${V3IO_ACCESS_KEY}

# Features
FEATURES_TABLE=netops_features

# Training
TRAIN_ON_LAST=7d
TRAIN_SIZE=0.7

# Parallelizem
NUMBER_OF_SHARDS=4

# Model
MODEL_FILENAME=netops.model
SAVE_TO=/v3io/bigdata/netops/models

%nuclio: setting 'V3IO_FRAMESD' environment variable
%nuclio: setting 'V3IO_USERNAME' environment variable
%nuclio: setting 'V3IO_ACCESS_KEY' environment variable
%nuclio: setting 'FEATURES_TABLE' environment variable
%nuclio: setting 'TRAIN_ON_LAST' environment variable
%nuclio: setting 'TRAIN_SIZE' environment variable
%nuclio: setting 'NUMBER_OF_SHARDS' environment variable
%nuclio: setting 'MODEL_FILENAME' environment variable
%nuclio: setting 'SAVE_TO' environment variable


%nuclio: cannot find "=" in line
%nuclio: cannot find "=" in line
%nuclio: cannot find "=" in line
%nuclio: cannot find "=" in line
%nuclio: cannot find "=" in line


## Function

### Imports

In [5]:
# Utils
import os
import time
import yaml
import pandas as pd
import datetime
import itertools

# DB Connection
import v3io_frames as v3f

# Parallelization
import dask.dataframe as dd
from dask.distributed import Client

# Function
import dask_ml.model_selection as dcv
from dask_ml.xgboost import XGBClassifier

### Helper functions

In [6]:
def get_data(context):
    df = context.v3f.read(backend='tsdb', query=f'select * from {context.features_table}',
                          start=f'now-{context.train_on_last}', end='now', multi_index=True)
    df = df.reset_index(drop=True)
    df = dd.from_pandas(df, npartitions=context.shards)
    return df

In [7]:
def get_train_test_sets_from_data(context, df: pd.DataFrame):
    drop_columns = [col for col in df.columns if 'is_error' in col]
    X = df.drop(drop_columns, axis=1)
    y = df.loc[:, 'is_error']
    X_train, X_test, y_train, y_test = dcv.train_test_split(X, y, train_size=context.train_size, test_size=1-context.train_size)
    return X_train, X_test, y_train, y_test

### Init context

In [8]:
def init_context(context):
    
    # Create our DB client
    v3io_client = v3f.Client(address='http://' + os.getenv('V3IO_FRAMESD', 'framesd:8081'), 
                        container='bigdata', 
                        password=os.environ['V3IO_ACCESS_KEY'], 
                        user=os.environ['V3IO_USERNAME'])
    setattr(context, 'v3f', v3io_client)
    
    # Create Dask client
    dask_client = Client()
    setattr(context, 'dask', dask_client)
    
    # Set time to train on
    train_on_last = os.getenv('TRAIN_ON_LAST', '7d')
    setattr(context, 'train_on_last', train_on_last)
    
    # Set training set size
    train_set_size = float(os.getenv('TRAIN_SIZE', 0.7))
    setattr(context, 'train_size', train_set_size)
    
    # Netops metrics table
    setattr(context, 'features_table', os.getenv('FEATURES_TABLE', 'netops_features'))
    
    # Dask shards / CV
    setattr(context, 'shards', int(os.getenv('NUMBER_OF_SHARDS', 4)))
    
    # Create save-to folder if needed
    model_filepath = os.getenv('SAVE_TO', '/v3io/bigdata/netops/models')
    if not os.path.exists(model_filepath):
        os.makedirs(model_filepath)
    setattr(context, 'model_filepath', os.path.join(model_filepath, 'netops.v1.model'))

### Handler

In [9]:
def handler(context, event):
    
    # Get data
    df = get_data(context) 

    # Split to Train / Test datasets
    X_train, X_test, y_train, y_test = get_train_test_sets_from_data(context, df)
    
    # Train
    model = XGBClassifier()
    model.fit(X_train, y_train)
    
    # Score
    score = model.score(X_test, y_test)
    print(score)
    
    # Save model
    model.save_model(context.model_filepath)

## Test

In [10]:
# nuclio: ignore
init_context(context)

/conda/lib/python3.6/site-packages/bokeh/themes/theme.py:131: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  json = yaml.load(f)
/conda/lib/python3.6/site-packages/distributed/bokeh/core.py:57: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn('\n' + msg)


In [11]:
# nuclio: ignore
# init_context(context)
event = nuclio.Event(body='')
output = handler(context, event)
output

0.8493765586034913


## Deployment

In [ ]:
%nuclio deploy -p netops -n training -c

%nuclio: ['deploy', '-p', 'netops', '-n', 'training', '-c', '/User/netops/tutorials/demos/netops/Nuclio-Training.ipynb']
%nuclio: [nuclio.deploy] 2019-05-15 14:42:00,377 (info) Building processor image
%nuclio: [nuclio.deploy] 2019-05-15 14:43:11,142 (info) Pushing image
%nuclio: [nuclio.deploy] 2019-05-15 14:43:37,406 (info) Build complete
%nuclio: [nuclio.deploy] 2019-05-15 14:43:43,468 (info) Function deploy complete
%nuclio: [nuclio.deploy] 2019-05-15 14:43:43,474 done updating training, function address: 3.122.56.83:32342
%nuclio: function deployed


# Test event

In [36]:
# nuclio: ignore
training_nuclio_event = {
    'experiment_metadata': {
        'id': 'id'
    },
    'input_data': 'FEATURES_TABLE',
    'input_type': 'TSDB / Parquet',
    'parameters': {
        # 'hyperparms'
    },
    'algorithm': 'SCIKIT_ENABLED_ALGORITHM',
    'output_model': 'URL / file path',
    'output_matrix': 'CV model training results - As CSV to output_model directory / url'
}

SyntaxError: invalid syntax (<ipython-input-36-704e0ddabbee>, line 6)

In [38]:
# nuclio: ignore
inference_nuclio_event = {
    'input_data_type': 'inline / file / tsdb',
    'input_data_path': 'tsdb path / file path',
    'feature_vector_if_inline_input': {
        'df': 'df json export'
    },
    'output_data_type': 'inline (as response json) / file / tsdb',
    'output_data_path': 'tsdb path / file path'
}